# DCM TO PNG CONVERSION
**WINDOW:** Soft Tissue

**WINDOW LEVEL/CENTER:** 40

**WINDOW WIDTH:** 400

In [ ]:
import os
import pydicom
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from PIL import Image

src_dir = r'src_path'
dst_dir = r'dst_path'

for folder_name in os.listdir(src_dir):
    folder = os.path.join(src_dir, folder_name)
    out_dir = os.path.join(dst_dir, folder_name)

    images = [os.path.join(folder, x) for x in os.listdir(folder)]
    names = os.listdir(folder)

    for i in tqdm(range(len(images))):
        di_image = pydicom.dcmread(images[i])
        def set_outside_scanner_to_air(raw_pixelarrays):
            # in OSIC we find outside-scanner-regions with raw-values of -2000. 
            # Let's threshold between air (0) and this default (-2000) using -1000
            raw_pixelarrays[raw_pixelarrays <= -1000] = 0
            return raw_pixelarrays


        def transform_to_hu(slices):
            images = np.stack([file.pixel_array for file in slices])
            images = images.astype(np.int16)

            images = set_outside_scanner_to_air(images)

            # convert to HU
            for n in range(len(slices)):

                intercept = slices[n].RescaleIntercept
                slope = slices[n].RescaleSlope

                if slope != 1:
                    images[n] = slope * images[n].astype(np.float64)
                    images[n] = images[n].astype(np.int16)

                images[n] += np.int16(intercept)

            return np.array(images, dtype=np.int16)


        hu_scans = transform_to_hu([di_image])

        def set_manual_window(hu_image, custom_center, custom_width):
            w_image = hu_image.copy()
            min_value = custom_center - (custom_width/2)
            max_value = custom_center + (custom_width/2)
            w_image[w_image < min_value] = min_value
            w_image[w_image > max_value] = max_value
            return w_image

        window_image = set_manual_window(hu_scans[0], 40, 400)

        output_filename = names[i].replace('.dcm', '.png')
        output_path = os.path.join(out_dir, output_filename)
        plt.imsave(output_path, window_image, cmap='gray')